In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import re

In [2]:
# read and store the master data file
df = pd.read_csv('MasterData.txt', sep = '|', 
                 parse_dates = ['CallConfirmedDT', 'UnitTransportedDT', 'UnitTransportedArrivalDT'], 
                 dtype = 'unicode')
#df['isALS_LOC'] = df['isALS_LOC'].astype(bool)

In [3]:
df.head()

,PrimaryKey,CallConfirmedDT,ShiftDay,TourOfShift,IncidentFirstDue,InitialIncidentType,DispatchedIncidentType,ArrivedIncidentType,FinalIncidentType,FinalIncidentTypeDescription,MutualAidReceived,ResponseUnitID,TransportUnitID,TransportUnitLOC,UnitPrimaryActionTaken,UnitOtherActionsTaken,UnitStation,UnitType,UnitArrivalOrder,UnitTransportedDT,UnitTransportedArrivalDT,PersonnelID,PersonnelRole,TransportLOC,TransportHospitalCode,TransportHospitalName,ReasonForChoosingHospital,PatientID,PatientAge,PrimaryImpression,SecondaryImpression,PatientDisposition,ProcedurePrimaryKey,ProcedurePerformed,MedicationPrimaryKey,MedicationGiven,isALS_LOC
0,1074428032,2019-05-28 00:45:44,C,3,442,ALS,ALS,ALS,ALS,ALS EMERGENCY,False,E442,M442,ALS-Paramedic,Provide manpower,NaN,42,Engine,1,NaT,NaT,6AD8C99E-9E01-E211-B5F5-78E7D18CFD3C,"Primary Patient Caregiver-At Scene,Primary Patient Caregiver-Transport",Level 3: 1 Provider (ALS Care),95,Reston Hospital Center,Patient's Choice,935111,30 - 39,Weakness (R53.1),"""Not Applicable""",Treated & Transported,1164292,CV - ECG - 12 Lead Obtained,NaN,NaN,False
1,1074428032,2019-05-28 00:45:44,C,3,442,ALS,ALS,ALS,ALS,ALS EMERGENCY,False,M442,M442,ALS-Paramedic,Provide advanced life support (ALS),Transport person,42,Medic,2,2019-05-28 01:08:07,2019-05-28 01:18:12,3ED8C99E-9E01-E211-B5F5-78E7D18CFD3C,"Driver-Response,Driver-Transport,Other Patient Caregiver-At Scene,Other Patient Caregiver-Transport",Level 3: 1 Provider (ALS Care),95,Reston Hospital Center,Patient's Choice,935111,30 - 39,Weakness (R53.1),"""Not Applicable""",Treated & Transported,1164288,IV Start - Extremity Vein (arm or leg),NaN,NaN,False
2,1074428032,2019-05-28 00:45:44,C,3,442,ALS,ALS,ALS,ALS,ALS EMERGENCY,False,M442,M442,ALS-Paramedic,Provide advanced life support (ALS),Transport person,42,Medic,2,2019-05-28 01:08:07,2019-05-28 01:18:12,3ED8C99E-9E01-E211-B5F5-78E7D18CFD3C,"Driver-Response,Driver-Transport,Other Patient Caregiver-At Scene,Other Patient Caregiver-Transport",Level 3: 1 Provider (ALS Care),95,Reston Hospital Center,Patient's Choice,935111,30 - 39,Weakness (R53.1),"""Not Applicable""",Treated & Transported,1164291,CV - ECG - 12 Lead Obtained,NaN,NaN,False
3,1074428032,2019-05-28 00:45:44,C,3,442,ALS,ALS,ALS,ALS,ALS EMERGENCY,False,M442,M442,ALS-Paramedic,Provide advanced life support (ALS),Transport person,42,Medic,2,2019-05-28 01:08:07,2019-05-28 01:18:12,6AD8C99E-9E01-E211-B5F5-78E7D18CFD3C,"Primary Patient Caregiver-At Scene,Primary Patient Caregiver-Transport",Level 3: 1 Provider (ALS Care),95,Reston Hospital Center,Patient's Choice,935111,30 - 39,Weakness (R53.1),"""Not Applicable""",Treated & Transported,1164288,IV Start - Extremity Vein (arm or leg),NaN,NaN,False
4,1074428032,2019-05-28 00:45:44,C,3,442,ALS,ALS,ALS,ALS,ALS EMERGENCY,False,M442,M442,ALS-Paramedic,Provide advanced life support (ALS),Transport person,42,Medic,2,2019-05-28 01:08:07,2019-05-28 01:18:12,6AD8C99E-9E01-E211-B5F5-78E7D18CFD3C,"Primary Patient Caregiver-At Scene,Primary Patient Caregiver-Transport",Level 3: 1 Provider (ALS Care),95,Reston Hospital Center,Patient's Choice,935111,30 - 39,Weakness (R53.1),"""Not Applicable""",Treated & Transported,1164291,CV - ECG - 12 Lead Obtained,NaN,NaN,False


In [4]:
print(f"The number of incidents is {df['PrimaryKey'].nunique()}.")
print(f"The number of patients is {df['PatientID'].nunique()}.")

The number of incidents is 121727.
The number of patients is 125032.


In [5]:
# patient age
age = df[['PrimaryKey', 'PatientID', 'PatientAge']].drop_duplicates()
print(age.head())

    PrimaryKey PatientID PatientAge
0   1074428032    935111    30 - 39
5   1074428058    935129    80 - 89
6   1074428490    926673    40 - 49
7   1074428563    925950    20 - 29
11  1074428574    929103    80 - 89


In [6]:
one_hot_age = pd.get_dummies(age.PatientAge, prefix='PAge')
combined = pd.concat([age, one_hot_age],axis = 1)
age_list = list(combined.columns[3:])
#select = combined[age_list]
re = combined.groupby(['PrimaryKey'])[age_list].apply(lambda x : x.sum())

In [7]:
age_order = ['PAge_< 1', 'PAge_1 - 9', 'PAge_10 - 19', 'PAge_20 - 29', 'PAge_30 - 39', 'PAge_40 - 49',
            'PAge_50 - 59', 'PAge_60 - 69', 'PAge_70 - 79', 'PAge_80 - 89', 'PAge_90 - 99', 'PAge_100 - 120']
re2 = re[age_order].reset_index()
re2

,PrimaryKey,PAge_< 1,PAge_1 - 9,PAge_10 - 19,PAge_20 - 29,PAge_30 - 39,PAge_40 - 49,PAge_50 - 59,PAge_60 - 69,PAge_70 - 79,PAge_80 - 89,PAge_90 - 99,PAge_100 - 120
0,1004010037,0,0,0,0,0,0,0,0,0,0,1,0
1,1004010043,0,0,0,0,0,0,0,0,0,1,0,0
2,1004010107,0,0,0,0,0,0,0,1,0,0,0,0
3,1004010111,0,0,0,0,0,0,0,0,0,1,0,0
4,1004010184,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121722,9994442192,0,0,0,0,0,0,0,0,0,1,0,0
121723,9994443610,0,0,0,0,0,0,0,0,1,0,0,0
121724,9994443961,0,0,0,0,0,0,0,0,1,0,0,0
121725,9994445708,0,0,0,0,0,0,1,0,0,0,0,0


In [8]:
# isALS_LOC
als = df[['PrimaryKey', 'isALS_LOC']].drop_duplicates()
t = als[als['isALS_LOC'] == 'True']
tt = set(t.PrimaryKey)
aa = set(als.PrimaryKey)
ff = aa.difference(tt)
pk_false_list = list(ff)
f = pd.DataFrame(pk_false_list, ['False' for i in range(len(pk_false_list))])
f = f.reset_index()
f.columns = ['isALS_LOC', 'PrimaryKey']
f = f[['PrimaryKey', 'isALS_LOC']]
als_loc = pd.concat([t, f])
als_loc

,PrimaryKey,isALS_LOC
5,1074428058,True
7,1074428563,True
35,1074440152,True
43,1074441305,True
57,1084013249,True
...,...,...
97898,1204093805,False
97899,1094107447,False
97900,1054316587,False
97901,1154191728,False


In [9]:
# MutualAidReceived
m_aid = df[['PrimaryKey', 'MutualAidReceived']].drop_duplicates()
t = m_aid[m_aid['MutualAidReceived'] == 'True']
tt = set(t.PrimaryKey)
aa = set(m_aid.PrimaryKey)
ff = aa.difference(tt)
pk_false_list = list(ff)
f = pd.DataFrame(pk_false_list, ['False' for i in range(len(pk_false_list))])
f = f.reset_index()
f.columns = ['MutualAidReceived', 'PrimaryKey']
f = f[['PrimaryKey', 'MutualAidReceived']]
aid = pd.concat([t, f])
aid

,PrimaryKey,MutualAidReceived
128,1074330660,True
134,1084154879,True
492,1074347957,True
1041,1074338289,True
1058,1074341764,True
...,...,...
120812,1204093805,False
120813,1094107447,False
120814,1054316587,False
120815,1154191728,False


In [88]:
import re
# Hospital Reason
#df.groupby('PrimaryKey')['ReasonForChoosingHospital'].nunique().value_counts()
reason = df[['PrimaryKey', 'ReasonForChoosingHospital']].drop_duplicates()
reason['ReasonForChoosingHospital'] = reason['ReasonForChoosingHospital'].astype(str)
def replace(g):
    return g.group(0).replace(',', '')
def remove_paran(text):
    return re.sub(r'\(.*?\)', replace, text)
reason['ReasonForChoosingHospital'] = reason['ReasonForChoosingHospital'].map(lambda x:remove_paran(x))
reason['ReasonForChoosingHospital'].str.split(',', expand=True)
reason = pd.concat([reason['PrimaryKey'], reason['ReasonForChoosingHospital'].str.split(',', expand=True)], axis = 1)
reason = pd.get_dummies(reason.iloc[:,1:],prefix='reason')
cols = ['reason_Closest Facility', 'reason_Diversion', 'reason_Family Choice', 'reason_Insurance Status/Requirement',
        'reason_Law Enforcement Choice', 'reason_On-Line/On-Scene Medical Direction', 'reason_Other',
        "reason_Patient's Choice", "reason_Patient's Physician's Choice", 'reason_Protocol',
        'reason_Specialty Center (Trauma STEMI Stroke)']
result = pd.DataFrame()
for col in cols:
    result[col] = reason[col].sum(axis = 1)
reason = df[['PrimaryKey', 'ReasonForChoosingHospital']].drop_duplicates()
result_fin = pd.concat([reason['PrimaryKey'], result], axis = 1)
result_fin = result_fin.groupby(['PrimaryKey'])[cols].apply(lambda x : x.sum())
result_fin

,reason_Closest Facility,reason_Diversion,reason_Family Choice,reason_Insurance Status/Requirement,reason_Law Enforcement Choice,reason_On-Line/On-Scene Medical Direction,reason_Other,reason_Patient's Choice,reason_Patient's Physician's Choice,reason_Protocol,reason_Specialty Center (Trauma STEMI Stroke)
PrimaryKey,,,,,,,,,,,
1004010037,0,0,1,0,0,0,0,0,0,0,0
1004010043,1,0,0,0,0,0,0,0,0,0,0
1004010107,1,0,0,0,0,0,0,0,0,0,0
1004010111,0,0,1,0,0,0,0,0,0,0,0
1004010184,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9994442192,1,0,0,0,0,0,0,0,0,0,0
9994443610,1,0,0,0,0,0,0,1,0,0,0
9994443961,0,0,0,0,0,0,0,1,0,0,0


In [98]:
df1 = re2
df2 = result_fin.reset_index()
df1.merge(df2, on = 'PrimaryKey').merge(als_loc, on = 'PrimaryKey').merge(aid, on = 'PrimaryKey')

,PrimaryKey,PAge_< 1,PAge_1 - 9,PAge_10 - 19,PAge_20 - 29,PAge_30 - 39,PAge_40 - 49,PAge_50 - 59,PAge_60 - 69,PAge_70 - 79,PAge_80 - 89,PAge_90 - 99,PAge_100 - 120,reason_Closest Facility,reason_Diversion,reason_Family Choice,reason_Insurance Status/Requirement,reason_Law Enforcement Choice,reason_On-Line/On-Scene Medical Direction,reason_Other,reason_Patient's Choice,reason_Patient's Physician's Choice,reason_Protocol,reason_Specialty Center (Trauma STEMI Stroke),isALS_LOC,MutualAidReceived
0,1004010037,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,False,False
1,1004010043,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,False,False
2,1004010107,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,True,False
3,1004010111,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,False,False
4,1004010184,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121722,9994442192,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,False,False
121723,9994443610,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,False,False
121724,9994443961,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,True,False
121725,9994445708,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,False,False


In [105]:
# why some columns have multiple indident date times
t = df[['PrimaryKey', 'CallConfirmedDT']].drop_duplicates()
t = t.groupby('PrimaryKey')['CallConfirmedDT'].nunique().reset_index()
t[t['CallConfirmedDT'] > 1]

,PrimaryKey,CallConfirmedDT
44,1004013459,2
232,1004040225,2
333,1004047623,3
426,1004052853,2
505,1004057238,2
...,...,...
121162,9984303228,2
121255,9994034407,2
121351,9994108009,2
121538,9994246562,2
